In [11]:
import os
import gzip
import json
import pickle
import numpy as np
from multiprocessing import Pool
import h5py
import myTelegram

In [2]:
# Paths to the data
main_path = '/net/big-tank/POOL/projects/fact/simulation/photon_stream/fact_tools/v.0.18.0/'
main_folder = [os.path.join(main_path, file) for file in os.listdir(main_path)]
main_folder

['/net/big-tank/POOL/projects/fact/simulation/photon_stream/fact_tools/v.0.18.0/gamma_gustav_12',
 '/net/big-tank/POOL/projects/fact/simulation/photon_stream/fact_tools/v.0.18.0/proton_12',
 '/net/big-tank/POOL/projects/fact/simulation/photon_stream/fact_tools/v.0.18.0/gamma_werner_12']

In [3]:
# Format dataset and labels to fit into tensorflow
def reformat(dataset, labels):
    dataset = dataset.reshape((-1, 46, 45, 1)).astype(np.float32)
    labels = (np.arange(2) == labels[:,None]).astype(np.float32)
    return dataset, labels

In [ ]:
# Load mapping-dict to switch from hexagonal to matrix
id_position = pickle.load(open("01_hexagonal_position_dict.p", "rb"))

data = []
count = 0
try:
# Iterate over all three folders
    for path in main_folder:

        # Iterate over every file in the folder
        for file in os.listdir(path):
            file_path = os.path.join(path, file)

            # Check if it is a json file
            if '.json' in file_path:

                # Open the file
                with gzip.open(file_path) as f:
                    count += 1

                    # Iterate over every line and get the data
                    for line in f:
                        line_data = json.loads(line.decode('utf-8'))
                        event_photons = line_data['PhotonArrivals_500ps']
                        night = line_data['Night']
                        run = line_data['Run']
                        event = line_data['Event']
                        fTotalEnergy = line_data['MCorsikaEvtHeader.fTotalEnergy']
                        zd_deg = line_data['Zd_deg']
                        az_deg = line_data['Az_deg']
                        trigger = line_data['Trigger']

                        label = True if 'gamma' in file_path else False

                        input_matrix = np.zeros([46,45])
                        for i in range(1440):
                            x, y = id_position[i]
                            input_matrix[int(x)][int(y)] = len(event_photons[i])
                        data.append([input_matrix, label, night, run, event, fTotalEnergy, zd_deg, az_deg, trigger])
                print(count, file)

        # Dezip the data and convert it to numpy-arrays
        pic, lab, night, run, event, fTotalEnergy, zd_deg, az_deg, trigger = zip(*data)
        pic, lab = reformat(np.array(pic), np.array(lab))
        night = np.array(night)
        run = np.array(run)
        event = np.array(event)
        fTotalEnergy = np.array(fTotalEnergy)
        zd_deg = np.array(zd_deg)
        az_deg = np.array(az_deg)
        trigger = np.array(trigger)

        # Store every original folder into a single h5 file
        with h5py.File('/fghfs/users/jbehnken/hdf5_files/'+path.rsplit('/', 1)[1]+'.h5', 'w') as hdf:
            hdf.create_dataset('Image', data=pic, compression='gzip', compression_opts=9)
            hdf.create_dataset('Label', data=lab, compression='gzip', compression_opts=9)
            hdf.create_dataset('Night', data=night, compression='gzip', compression_opts=9)
            hdf.create_dataset('Run', data=run, compression='gzip', compression_opts=9)
            hdf.create_dataset('Event', data=event, compression='gzip', compression_opts=9)
            hdf.create_dataset('fTotalEnergy', data=fTotalEnergy, compression='gzip', compression_opts=9)
            hdf.create_dataset('Zd_deg', data=zd_deg, compression='gzip', compression_opts=9)
            hdf.create_dataset('Az_deg', data=az_deg, compression='gzip', compression_opts=9)
            hdf.create_dataset('Trigger', data=trigger, compression='gzip', compression_opts=9)

        message = path.rsplit('/', 1)[1]+' has been processed.'
        myTelegram.sendTelegram(message)
except:
    message = 'Leider ist ein Fehler aufgetreten.'
    myTelegram.sendTelegram(message)
    raise
finally:
    message = 'Berechnung wurde beendet.'
    myTelegram.sendTelegram(message)

output_spe_extractor_mc_39.json
output_spe_extractor_mc_72.json
output_spe_extractor_mc_43.json
output_spe_extractor_mc_83.json
output_spe_extractor_mc_21.json
